# Fase de Modelización 

## Carga del modelo MovilNet

En este primer punto vamos a cargar el modelo MovilNet para iniciar la fase de entrenamiento del modelo con los datos obtenidos en la fase de preanalisis y resolución. Luego probaremos una segunda parte con una CNN creada desde cero para contrastar resultados

### 1. Importamos las librerias que necesitamos para instanciar el modelo 

In [3]:
from IPython.display import display, Javascript

display(Javascript('IPython.notebook.kernel.restart()'))

<IPython.core.display.Javascript object>

In [2]:
%run imports.py

In [3]:
df_model = pd.read_csv("output\set_aumentado_v01.csv")
df_model.shape


(52230, 4)

In [5]:
def image_label_load(data): 
    images = []
    labels = []

    for image_path, label in tqdm(zip(data['path_files'].tolist(), data['label'].tolist()), desc="Codificando imagenes"):
        try: 
            img = Image.open(image_path).resize((244,244)).convert('RGB')
            images.append(np.array(img))
            labels.append(np.array(label))
        except Exception as e: 
            print(f"{e} Levantado tras ejecutar la ruta {image_path}")
    
    return np.array(images), np.array(labels)

images, labels = image_label_load(df_model)

# Codificar las etiquetas
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = to_categorical(labels_encoded, num_classes=5)  # Asegúrate de especificar 5 clases

print(f"Shape of images: {images.shape}")
print(f"Shape of labels: {labels_encoded.shape}")

Codificando imagenes: 52230it [07:46, 112.05it/s]


Shape of images: (52230, 244, 244, 3)
Shape of labels: (52230, 5)


In [6]:
# Carga del MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(244,244,3))

# Congelar capas base del modelo -> Conserva los pesos establecidos por el aprendizaje de imagenet
base_model.trainable = False 

#Añadimos capas superiores para ajustar el modelo a nuestro caso concreto: 

x = base_model.output 
x = GlobalAveragePooling2D()(x) 
x = Dense(128, activation='relu')(x) 

predictions = Dense(5, activation='softmax')(x) # Ajuste para 5 clases (menu, outside, drink, inside, food)

# Instancia del MobileNetV2 + capas superiores 
model = Model(inputs=base_model.input, outputs=predictions)

# Compilacion del modelo en conjunto: 

model.compile(optimizer ='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


C:\Users\varel\AppData\Local\Temp\ipykernel_6648\3325738747.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(244,244,3))


In [7]:
# Train/Test split -> Division  de conjuntos de datos en entrenamiento y validacion 

X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.25, random_state=41)


# Data Augmentation --> para ganar mas variedad de datos a la hora de entrenar 
"""
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
"""
                ### HIPER-PARAMETROS DE DATA AUGMENTATION PARA ENTRENAMIENTO ###
datagen = ImageDataGenerator()
# Ajustamos el generador de datos al conjunto de entrenamiento 

datagen.fit(X_train)

MemoryError: Unable to allocate 26.1 GiB for an array with shape (39172, 244, 244, 3) and data type float32

In [ ]:
# Numero de pasos por epoch
batch_size = 32
steps_per_epoch = len(X_train) // batch_size
# Entrenar el modelo
history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch,
                    epochs=10,
                    validation_data=(X_test, y_test))

Epoch 1/10


c:\Users\varel\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


445/445 ━━━━━━━━━━━━━━━━━━━━ 572s 1s/step - accuracy: 0.7466 - loss: 0.7189 - val_accuracy: 0.8042 - val_loss: 0.5134
Epoch 2/10
  1/445 ━━━━━━━━━━━━━━━━━━━━ 6:40 902ms/step - accuracy: 0.9375 - loss: 0.3731

c:\Users\varel\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


445/445 ━━━━━━━━━━━━━━━━━━━━ 135s 302ms/step - accuracy: 0.9375 - loss: 0.3731 - val_accuracy: 0.7876 - val_loss: 0.5470
Epoch 3/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 541s 1s/step - accuracy: 0.8341 - loss: 0.4596 - val_accuracy: 0.8276 - val_loss: 0.4660
Epoch 4/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 135s 303ms/step - accuracy: 0.8125 - loss: 0.4949 - val_accuracy: 0.8331 - val_loss: 0.4631
Epoch 5/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 542s 1s/step - accuracy: 0.8539 - loss: 0.4069 - val_accuracy: 0.8257 - val_loss: 0.4751
Epoch 6/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 135s 301ms/step - accuracy: 0.8750 - loss: 0.4282 - val_accuracy: 0.8238 - val_loss: 0.4819
Epoch 7/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 543s 1s/step - accuracy: 0.8639 - loss: 0.3794 - val_accuracy: 0.8440 - val_loss: 0.4356
Epoch 8/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 134s 300ms/step - accuracy: 0.8438 - loss: 0.2885 - val_accuracy: 0.8421 - val_loss: 0.4408
Epoch 9/10
445/445 ━━━━━━━━━━━━━━━━━━━━ 544s 1s/step - accuracy: 0.8731 - loss: 0.3483 - val_acc

In [8]:
# Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)


# Calcular el F1-score
f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
print(f'F1-score: {f1}')

# Calcular el accuracy
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f'Accuracy: {accuracy}')

# Generar el reporte de clasificación
report = classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_)
print(report)

# Generar la curva ROC y el AUC para cada clase
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(5):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotear todas las curvas ROC
plt.figure()
colors = ['aqua', 'darkorange', 'cornflowerblue', 'green', 'red']
for i, color in zip(range(5), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label=f'ROC curve (area = {roc_auc[i]:.2f}) for class {i}')
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0]) # apuntar 
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

 70/409 ━━━━━━━━━━━━━━━━━━━━ 5:52 1s/step